# Generate features by composition 

In [1]:
import pandas as pd

# get data of mp/exp from the file 
mp_df = pd.read_csv("./Data/composition_data/mp_band_gap.csv")
exp_df = pd.read_csv("./Data/composition_data/exp_band_gap.csv")

In [2]:
mp_df.shape, exp_df.shape

((5472, 2), (1930, 2))

## Feature Engineering

### 1. 基于元素组成进行特征化

In [3]:
# Change the name of composition column
mp_df.rename(columns={'composition': 'composition_str'}, inplace=True)
exp_df.rename(columns={'composition': 'composition_str'}, inplace=True)

In [4]:
mp_df.head()

,composition_str,band_gap
0,Ag8B48Cl48,2.7028
1,Ag40Te16Br12,0.8722
2,Ag30P8S32Cl6,1.2888
3,Ag4C2O6,0.4736
4,Ag12Ge6S36O126,2.8072


In [5]:
# transform the composition_str column to composition column
from matminer.featurizers.conversions import StrToComposition

stc = StrToComposition(target_col_id='composition')
mp_df = stc.featurize_dataframe(mp_df, col_id="composition_str", pbar=True)

StrToComposition:   0%|          | 0/5472 [00:00<?, ?it/s]

In [6]:
exp_df = stc.featurize_dataframe(exp_df, col_id="composition_str", pbar=True)

StrToComposition:   0%|          | 0/1930 [00:00<?, ?it/s]

In [7]:
mp_df.head()

,composition_str,band_gap,composition
0,Ag8B48Cl48,2.7028,"(Ag, B, Cl)"
1,Ag40Te16Br12,0.8722,"(Ag, Te, Br)"
2,Ag30P8S32Cl6,1.2888,"(Ag, P, S, Cl)"
3,Ag4C2O6,0.4736,"(Ag, C, O)"
4,Ag12Ge6S36O126,2.8072,"(Ag, Ge, S, O)"


In [8]:
exp_df.head()

,composition_str,band_gap,composition
0,Cu3Sb1Se4,0.40,"(Cu, Sb, Se)"
1,Zn1In2S4,2.68,"(Zn, In, S)"
2,K2Cd3Te4,2.26,"(K, Cd, Te)"
3,In1Sb1,0.22,"(In, Sb)"
4,K2Cu1Nb1S4,2.82,"(K, Cu, Nb, S)"


In [10]:
# composition-based features
from matminer.featurizers.composition import ElementProperty
featurizer = ElementProperty.from_preset('magpie') 	# extracting features from a predefined database， e.g. magpie, jarvis, etc.
mp_featurized_df = featurizer.featurize_dataframe(mp_df, col_id='composition')

/home/codespace/.python/current/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


ElementProperty:   0%|          | 0/5472 [00:00<?, ?it/s]

In [11]:
mp_featurized_df.head()

,composition_str,band_gap,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,Ag8B48Cl48,2.7028,"(Ag, B, Cl)",5.0,47.0,42.0,13.769231,8.094675,5.0,65.0,...,0.0,0.0,0.0,0.0,64.0,225.0,161.0,123.461538,54.887574,64.0
1,Ag40Te16Br12,0.8722,"(Ag, Te, Br)",35.0,52.0,17.0,46.058824,3.903114,47.0,65.0,...,0.0,0.0,0.0,0.0,64.0,225.0,161.0,179.411765,53.633218,225.0
2,Ag30P8S32Cl6,1.2888,"(Ag, P, S, Cl)",15.0,47.0,32.0,28.210526,14.833795,16.0,65.0,...,0.0,0.0,0.0,0.0,2.0,225.0,223.0,123.552632,80.090028,70.0
3,Ag4C2O6,0.4736,"(Ag, C, O)",6.0,47.0,41.0,20.666667,17.555556,8.0,65.0,...,0.0,0.0,0.0,0.0,12.0,225.0,213.0,113.333333,101.333333,12.0
4,Ag12Ge6S36O126,2.8072,"(Ag, Ge, S, O)",8.0,47.0,39.0,13.000000,7.000000,8.0,65.0,...,0.0,0.0,0.0,0.0,12.0,225.0,213.0,44.900000,46.060000,12.0


In [12]:
exp_featurized_df = featurizer.featurize_dataframe(exp_df, col_id='composition')
exp_featurized_df.head()

ElementProperty:   0%|          | 0/1930 [00:00<?, ?it/s]

,composition_str,band_gap,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,Cu3Sb1Se4,0.40,"(Cu, Sb, Se)",29.0,51.0,22.0,34.250000,4.187500,34.0,64.0,...,0.0,0.0,0.0,0.0,14.0,225.0,211.0,112.125000,98.125000,14.0
1,Zn1In2S4,2.68,"(Zn, In, S)",16.0,49.0,33.0,27.428571,13.061224,16.0,69.0,...,0.0,0.0,0.0,0.0,70.0,194.0,124.0,107.428571,42.775510,70.0
2,K2Cd3Te4,2.26,"(K, Cd, Te)",19.0,52.0,33.0,43.333333,10.814815,52.0,3.0,...,0.0,0.0,0.0,0.0,152.0,229.0,77.0,183.111111,27.654321,152.0
3,In1Sb1,0.22,"(In, Sb)",49.0,51.0,2.0,50.000000,1.000000,49.0,75.0,...,0.0,0.0,0.0,0.0,139.0,166.0,27.0,152.500000,13.500000,139.0
4,K2Cu1Nb1S4,2.82,"(K, Cu, Nb, S)",16.0,41.0,25.0,21.500000,6.750000,16.0,3.0,...,0.0,0.0,0.0,0.0,70.0,229.0,159.0,149.000000,79.000000,70.0


In [13]:
mp_featurized_df.isnull().any().sum()

np.int64(0)

In [14]:
exp_featurized_df.isnull().any().sum()

np.int64(0)

In [15]:
mp_featurized_df.drop(columns=['composition'], inplace=True)    
mp_featurized_df.rename(columns={'composition_str': 'composition'}, inplace=True)
mp_featurized_df.head()

,composition,band_gap,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,Ag8B48Cl48,2.7028,5.0,47.0,42.0,13.769231,8.094675,5.0,65.0,94.0,...,0.0,0.0,0.0,0.0,64.0,225.0,161.0,123.461538,54.887574,64.0
1,Ag40Te16Br12,0.8722,35.0,52.0,17.0,46.058824,3.903114,47.0,65.0,95.0,...,0.0,0.0,0.0,0.0,64.0,225.0,161.0,179.411765,53.633218,225.0
2,Ag30P8S32Cl6,1.2888,15.0,47.0,32.0,28.210526,14.833795,16.0,65.0,94.0,...,0.0,0.0,0.0,0.0,2.0,225.0,223.0,123.552632,80.090028,70.0
3,Ag4C2O6,0.4736,6.0,47.0,41.0,20.666667,17.555556,8.0,65.0,87.0,...,0.0,0.0,0.0,0.0,12.0,225.0,213.0,113.333333,101.333333,12.0
4,Ag12Ge6S36O126,2.8072,8.0,47.0,39.0,13.000000,7.000000,8.0,65.0,88.0,...,0.0,0.0,0.0,0.0,12.0,225.0,213.0,44.900000,46.060000,12.0


In [16]:
exp_featurized_df.drop(columns=['composition'], inplace=True)
exp_featurized_df.rename(columns={'composition_str': 'composition'}, inplace=True)
exp_featurized_df.head()

,composition,band_gap,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,Cu3Sb1Se4,0.40,29.0,51.0,22.0,34.250000,4.187500,34.0,64.0,89.0,...,0.0,0.0,0.0,0.0,14.0,225.0,211.0,112.125000,98.125000,14.0
1,Zn1In2S4,2.68,16.0,49.0,33.0,27.428571,13.061224,16.0,69.0,88.0,...,0.0,0.0,0.0,0.0,70.0,194.0,124.0,107.428571,42.775510,70.0
2,K2Cd3Te4,2.26,19.0,52.0,33.0,43.333333,10.814815,52.0,3.0,90.0,...,0.0,0.0,0.0,0.0,152.0,229.0,77.0,183.111111,27.654321,152.0
3,In1Sb1,0.22,49.0,51.0,2.0,50.000000,1.000000,49.0,75.0,85.0,...,0.0,0.0,0.0,0.0,139.0,166.0,27.0,152.500000,13.500000,139.0
4,K2Cu1Nb1S4,2.82,16.0,41.0,25.0,21.500000,6.750000,16.0,3.0,88.0,...,0.0,0.0,0.0,0.0,70.0,229.0,159.0,149.000000,79.000000,70.0


In [17]:
# save the data to csv file
import os
file_path = os.path.join(os.getcwd(), "./Data/featured_data")
os.makedirs(file_path, exist_ok=True)
mp_featurized_df.to_csv(os.path.join(file_path, 'mp_elements_feat.csv'), index=False)
exp_featurized_df.to_csv(os.path.join(file_path, 'exp_elements_feat.csv'), index=False)

### 2. MP数据加入其他物理特征

In [18]:
# load the data
mp_eda_df = pd.read_csv('./Data/mp_eda_data.csv')

In [19]:
mp_eda_df.head()

,material_id,composition,nsites,crystal_system,space_group,volume,density,density_atomic,efermi,energy_per_atom,formation_energy_per_atom,is_gap_direct,band_gap,is_oxide
0,mp-32800,Ac16 S24,40,1,122,1118.407852,6.535149,27.960196,5.791820,-34.768478,-2.492486,0.0,2.2962,0.0
1,mp-867311,Ac1 Ag1 Te2,4,0,225,122.518406,7.997421,30.629602,6.031096,-36.203183,-0.996232,0.0,0.0794,0.0
2,mp-866101,Ac1 Cr1 O3,5,0,221,61.362845,8.848788,12.272569,6.364737,-8.862593,-3.138972,0.0,2.0031,1.0
3,mp-861502,Ac1 Fe1 O3,5,0,221,61.797311,8.889999,12.359462,6.509045,-8.258555,-2.771539,0.0,0.9888,1.0
4,mp-1183053,Ac1 Ga1 O3,5,0,221,61.455078,9.314495,12.291016,5.476935,-7.461883,-3.063253,0.0,2.8959,1.0


In [20]:
mp_eda_df.rename(columns={'composition': 'composition_str'}, inplace=True)

In [21]:
from matminer.featurizers.conversions import StrToComposition

stc = StrToComposition(target_col_id='composition')
mp_eda_df = stc.featurize_dataframe(mp_eda_df, col_id="composition_str", pbar=True)

StrToComposition:   0%|          | 0/10342 [00:00<?, ?it/s]

In [22]:
# composition-based features
from matminer.featurizers.composition import ElementProperty
featurizer = ElementProperty.from_preset('magpie') 	# extracting features from a predefined database， e.g. magpie, jarvis, etc.
mp_featurized_df2 = featurizer.featurize_dataframe(mp_eda_df, col_id='composition')

/home/codespace/.python/current/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


ElementProperty:   0%|          | 0/10342 [00:00<?, ?it/s]

In [23]:
mp_featurized_df2.head()    

,material_id,composition_str,nsites,crystal_system,space_group,volume,density,density_atomic,efermi,energy_per_atom,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,mp-32800,Ac16 S24,40,1,122,1118.407852,6.535149,27.960196,5.791820,-34.768478,...,0.000000,0.000000,0.000000,0.0,70.0,225.0,155.0,132.0,74.4,70.0
1,mp-867311,Ac1 Ag1 Te2,4,0,225,122.518406,7.997421,30.629602,6.031096,-36.203183,...,0.000000,0.000000,0.000000,0.0,152.0,225.0,73.0,188.5,36.5,152.0
2,mp-866101,Ac1 Cr1 O3,5,0,221,61.362845,8.848788,12.272569,6.364737,-8.862593,...,0.000000,0.000000,0.000000,0.0,12.0,229.0,217.0,98.0,103.2,12.0
3,mp-861502,Ac1 Fe1 O3,5,0,221,61.797311,8.889999,12.359462,6.509045,-8.258555,...,2.110663,0.422133,0.675412,0.0,12.0,229.0,217.0,98.0,103.2,12.0
4,mp-1183053,Ac1 Ga1 O3,5,0,221,61.455078,9.314495,12.291016,5.476935,-7.461883,...,0.000000,0.000000,0.000000,0.0,12.0,225.0,213.0,65.0,64.0,12.0


In [24]:
mp_featurized_df2.isnull().any().sum()

np.int64(0)

In [25]:
mp_featurized_df2['composition_str'] = mp_featurized_df2['composition']
mp_featurized_df2.drop(columns=['composition'], inplace=True)    
mp_featurized_df2.rename(columns={'composition_str': 'composition'}, inplace=True)
mp_featurized_df2['composition'] = mp_featurized_df2['composition'].apply(lambda x: str(x).replace(" ", ""))
mp_featurized_df2.head()

,material_id,composition,nsites,crystal_system,space_group,volume,density,density_atomic,efermi,energy_per_atom,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,mp-32800,Ac16S24,40,1,122,1118.407852,6.535149,27.960196,5.791820,-34.768478,...,0.000000,0.000000,0.000000,0.0,70.0,225.0,155.0,132.0,74.4,70.0
1,mp-867311,Ac1Ag1Te2,4,0,225,122.518406,7.997421,30.629602,6.031096,-36.203183,...,0.000000,0.000000,0.000000,0.0,152.0,225.0,73.0,188.5,36.5,152.0
2,mp-866101,Ac1Cr1O3,5,0,221,61.362845,8.848788,12.272569,6.364737,-8.862593,...,0.000000,0.000000,0.000000,0.0,12.0,229.0,217.0,98.0,103.2,12.0
3,mp-861502,Ac1Fe1O3,5,0,221,61.797311,8.889999,12.359462,6.509045,-8.258555,...,2.110663,0.422133,0.675412,0.0,12.0,229.0,217.0,98.0,103.2,12.0
4,mp-1183053,Ac1Ga1O3,5,0,221,61.455078,9.314495,12.291016,5.476935,-7.461883,...,0.000000,0.000000,0.000000,0.0,12.0,225.0,213.0,65.0,64.0,12.0


In [26]:
mp_featurized_df2.drop(columns=['material_id'], inplace=True)

In [27]:
# save the data to csv file
mp_featurized_df2.to_csv(os.path.join(file_path, 'mp_elements_plus_feat.csv'), index=False)